In [1]:
# 화면에서 검색어를 입력받아
# selenium으로 키워드 검색
# 검색결과를 수집( 단일 영상 타입)
# - 채널명, 영상제목, 영상url, 조회수, 영상등록날짜
# - 'h3',{'class':'title-and-badge style-scope ytd-video-renderer'}
# 검색결과를 수집( 재생목록 타입)
# - 'div',{'class':'style-scope ytd-playlist-renderer'}
# - 채널명, 재생목록제목, 재생목록url
# 데이터 프레임 만들기
# - 단일영상타입 데이터 프레임
# - 재생목록타입 데이터 프레임

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import pandas as pd

In [2]:
driver = wb.Chrome('C:/Users/학생/0.pjha/chromedriver')

In [3]:
driver.get('http://youtube.com')

In [4]:
driver.find_element_by_id('search').send_keys('selenium')

In [5]:
driver.find_element_by_id('search-icon-legacy').click()

In [6]:
source = driver.page_source

In [7]:
soup = bs(source, 'html.parser')

In [18]:
b = soup.find_all('ytd-item-section-renderer',{'class':'style-scope ytd-section-list-renderer'})
b[2]

<ytd-item-section-renderer class="style-scope ytd-section-list-renderer" use-height-hack=""><!--css-build:shady--><div class="style-scope ytd-item-section-renderer" id="header"></div>
<div class="style-scope ytd-item-section-renderer" id="spinner-container">
<tp-yt-paper-spinner-lite aria-hidden="true" class="style-scope ytd-item-section-renderer"><!--css-build:shady--><div class="style-scope tp-yt-paper-spinner-lite" id="spinnerContainer"><div class="spinner-layer style-scope tp-yt-paper-spinner-lite"><div class="circle-clipper left style-scope tp-yt-paper-spinner-lite"><div class="circle style-scope tp-yt-paper-spinner-lite"></div></div><div class="circle-clipper right style-scope tp-yt-paper-spinner-lite"><div class="circle style-scope tp-yt-paper-spinner-lite"></div></div></div></div></tp-yt-paper-spinner-lite>
</div>
<div class="style-scope ytd-item-section-renderer" id="contents"><ytd-video-renderer class="style-scope ytd-item-section-renderer" inline-title-icon="" lockup="true" 

In [9]:
single_type = soup.find_all('h3',{'class':'title-and-badge style-scope ytd-video-renderer'}) 

In [10]:
single_type[0].find('a')['aria-label'].split(' 게')[0]

'파이썬 자동화 selenium 사용법 브라우저 매크로 프로그램 만들기'

In [11]:
single_type[0].find('a')['href']

'/watch?v=ZFmTwbRQ0uc'

In [12]:
len(single_type)

20

In [13]:
list_type = soup.find_all('div',{'class':'style-scope ytd-playlist-renderer'})
len(list_type)

4

In [14]:
searchlist_list_type_title = []
searchlist_list_type_channel = []
searchlist_list_type_url = []
for i in range(len(list_type)):
    searchlist_list_type_title.append(list_type[i].find('span')['title'])
    searchlist_list_type_channel.append(list_type[i].find('div',{'id':'tooltip'}).text.replace('\n','').replace(' ',''))
    list_type_url = 'https://www.youtube.com' + list_type[i].find('a')['href']
    searchlist_list_type_url.append(list_type_url)

In [15]:
list_df = pd.DataFrame(list(zip(searchlist_list_type_title,searchlist_list_type_channel,
                               searchlist_list_type_url)),columns=['리스트제목',
                                                                  '채널명','url'])
list_df

,리스트제목,채널명,url
0,Python Selenium Tutorials,TechWithTim,https://www.youtube.com/watch?v=Xjv1sY630Uc&li...
1,Python+Selenium入门教学,Inty,https://www.youtube.com/watch?v=o6yzNaRAzW8&li...
2,파이썬 selenium(셀레니움) 강의,프로그래머김플스튜디오,https://www.youtube.com/watch?v=j_BW5vNrcxA&li...
3,Selenium with Python,SDET-QAAutomationTechie,https://www.youtube.com/watch?v=IYILCEV5j6s&li...


In [16]:
searchlist_single_type_channel = []
searchlist_single_type_title = []
searchlist_single_type_date = []
searchlist_single_type_clicks = []
searchlist_single_type_url = []
for i in range(len(single_type)):
    searchlist_single_type_channel.append(soup.find_all('div',{'id':'channel-info'})[i].find('a',{'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text)
    searchlist_single_type_title.append(single_type[i].find('a')['aria-label'].split(' 게')[0])
    url = 'https://www.youtube.com'+single_type[i].find('a')['href']
    searchlist_single_type_url.append(url)
for i in range(0,38):
    if i%2 == 1:
        searchlist_single_type_date.append(b[1].find_all('span',{'class':'style-scope ytd-video-meta-block'})[i].text)
    else:
        searchlist_single_type_clicks.append(b[1].find_all('span',{'class':'style-scope ytd-video-meta-block'})[i].text)

single_df = pd.DataFrame(list(zip(searchlist_single_type_title,searchlist_single_type_channel,
                                searchlist_single_type_url,searchlist_single_type_date,searchlist_single_type_clicks)),
                             columns=['제목','채널','url','날짜','조회수'])
single_df

IndexError: list index out of range